In [152]:
import serial
import sys
import time

In [153]:
def listSerialPorts():
    """ 
    Lists serial port names

        :raises EnvironmentError:
            On unsupported or unknown platforms
        :returns:
            A list of the serial ports available on the system
    """
    if sys.platform.startswith('win'):
        ports = ['COM%s' % (i + 1) for i in range(256)]
    elif sys.platform.startswith('linux') or sys.platform.startswith('cygwin'):
        # this excludes your current terminal "/dev/tty"
        ports = glob.glob('/dev/tty[A-Za-z]*')
    elif sys.platform.startswith('darwin'):
        ports = glob.glob('/dev/tty.*')
    else:
        raise EnvironmentError('Unsupported platform')

    result = []
    for port in ports:
        try:
            s = serial.Serial(port)
            s.close()
            result.append(port)
        except (OSError, serial.SerialException):
            pass
    return result

In [154]:
listSerialPorts()

['COM3']

In [155]:
port = serial.Serial('COM3', 115200, timeout=0.1)

In [156]:
def readAll(port):
    time.sleep(0.001)
    message = ''
    while port.inWaiting():
        message += port.read(1).decode("utf-8")
    
    return message

In [157]:
readAll(port)

''

In [158]:
def write(port, expression, eol):
    port.flushInput()
    expression = expression.strip()
    expression = expression + eol
    expr_enc = expression.encode()
    port.write(expr_enc)

In [159]:
write(port, 'M114', '\n')

In [160]:
write(port, 'G0 Z0', '\n')

readAll(port)

'ok\r\n'

In [161]:
readAll(port)

''

In [162]:
write(port, 'M400', '\n')
readAll(port)

''

In [173]:
write(port, 'G0 B0', '\n')

readAll(port)

'ok\r\n'